# **LifeController**

Il **LifeController** è una componente ipotetica usata per gestire i **punti vita (HP)** di un oggetto di scena in Unity. Il suo compito è controllare la vita dell’oggetto e cosa accade quando questa arriva a zero.

## **1. Enum per gestire il comportamento alla sconfitta**

Si definisce un **enum** dentro la classe:

```csharp
public enum ON_DEFEAT_BEHAVIOUR
{
    DISABLE,
    DESTROY,
    NONE
}
```

E una variabile esposta nell’Inspector decide cosa fare quando gli HP arrivano a 0:

* **DISABLE** → `gameObject.SetActive(false);`
* **DESTROY** → `Destroy(gameObject);`
* **NONE** → non fare nulla (utile in casi particolari)

## **2. Funzione SetHP(int hp)**

Serve per assegnare un valore agli HP:

* fa un **clamp** tra 0 e HP massimi
* se gli HP scendono a 0, esegue il comportamento scelto nell’enum

## **3. Funzione AddHP(int amount)**

Comoda per modificare la vita senza calcolare ogni volta:

```csharp
AddHP(amount)
```

Internamente richiama:

```csharp
SetHP(GetHP() + amount);
```

Esempio: curare di 5 punti → `lifeController.AddHP(5);`

## **4. Gestione del valore massimo della vita (MaxHP)**

Si può aggiungere una **boolean** per decidere se lo Start() deve impostare automaticamente gli HP al massimo.

La funzione **SetMaxHp(int maxHp)** deve decidere come comportarsi:

* se aumentano gli HP massimi → aumentano anche quelli attuali?
* oppure si mantiene la **proporzione**? (ex: ero al 50%, resto al 50%)
* cosa succede se il massimo diminuisce?
* ogni volta che cambia, bisogna verificare che gli HP attuali non superino il nuovo massimo

Queste decisioni dipendono dal **game design**.

# **In sintesi**

Il *LifeController* introduce:

* un **enum** per gestire diversi comportamenti alla morte dell’oggetto
* funzioni per **impostare** e **aggiornare** gli HP (`SetHP`, `AddHP`)
* logiche opzionali per la gestione del **MaxHP**
* un uso corretto di **variabili private**, clamp dei valori e funzioni esposte pubblicamente

È un esercizio per unire concetti base di Unity: enum, Inspector, metodi, variabili, design del comportamento.

---



# **Danno da Contatto**

Per capire quando un nemico (o qualsiasi oggetto) viene colpito, possiamo usare:

* **OnCollisionEnter2D()**
* **OnTriggerEnter2D()**

Quando avviene il contatto, si prende il **collider** dell’oggetto colpito e si prova a cercare al suo interno un **LifeController**.

Esempio:

```csharp
LifeController life = collider.GetComponent<LifeController>();
if (life != null)
{
    life.AddHp(-damage);   // sottrae vita
}
```

Significa:

* Se l’oggetto colpito **ha un LifeController**, allora subisce danno.
* Il danno è applicato usando **AddHP()** con un valore negativo.

---

# **Layers & LayerMask**

## **1. Cosa sono i Layers**

I **Layers** sono un sistema per organizzare gli oggetti della scena in gruppi che Unity può usare per:

* decidere **quali oggetti collidevano tra loro**
* decidere **quali oggetti una Camera può vedere**
* filtrare quali collider considerare nei controlli di fisica

Un GameObject può appartenere **ad un solo Layer**.

Unity fornisce alcuni layer predefiniti, ma possiamo crearne di nuovi fino a un massimo di **32 Layers**.

## **2. Uso dei Layers nella Camera (Culling Mask)**

Nella Camera troviamo la proprietà **Culling Mask**, che serve per scegliere **quali Layers la Camera deve visualizzare**.

Esempi:

* nascondere l’HUD alla camera di una mini-mappa
* far vedere solo certi oggetti a una camera di debug
* escludere effetti particolari (es. layer “Effects”)

La Culling Mask funziona come un filtro basato sui Layers.

## **3. Collisioni tra Layers (Physics & Physics2D)**

In Unity possiamo definire **quali Layers possono collidere tra loro**:

**Menu:**
`Edit > Project Settings > Physics` *(o Physics 2D)*

Qui appare una **matrice**:

* righe = Layers
* colonne = Layers
* una spunta indica che quei due layers possono collidere

Questo permette ottimizzazione e controllo del gameplay, ad esempio:

* PlayerBullet collide solo con Enemy
* EnemyBullet collide solo con Player
* BulletPass non collide con niente (spunta tolta)

## **4. LayerMask nei codici**

Un **LayerMask** è un tipo di dato che rappresenta una combinazione di Layers (come la Culling Mask della Camera).

Serve per filtrare quali oggetti la fisica deve considerare, ad esempio:

* un raggio che colpisce solo alcuni oggetti
* un’area di danno che ignora il Player
* una funzione di ricerca che restituisce solo certi layer

È lo stesso concetto usato dalla Camera, ma applicato nel codice.

Esempio classico:

```csharp
RaycastHit2D hit = Physics2D.Raycast(origin, direction, distance, layerMask);
```

Solo i Layers inclusi nel `layerMask` verranno considerati dal raycast.

# **In sintesi**

* I **Layers** sono categorie assegnabili agli oggetti.
* La **Camera** usa i Layers attraverso la proprietà **Culling Mask** per decidere cosa vedere.
* Il **motore fisico** usa la matrice delle collisioni per decidere quali Layers interagiscono.
* Nel codice si usa **LayerMask** per filtrare oggetti nelle funzioni fisiche (raycast, overlap, ecc.).

---


# **Rate of Fire & Spawn dei Proiettili**

Per far sparare un nemico a intervalli regolari servono **due cose**:

1. **Un intervallo di tempo tra uno sparo e l'altro** (`interval`)
2. **Un meccanismo per capire quando è il momento di sparare**

Unity offre **due approcci equivalenti**.

# **Approccio 1 – Contare il tempo che passa (Time.deltaTime)**

Incrementiamo un timer ogni frame:

```csharp
timer += Time.deltaTime;

if (timer > interval)
{
    timer -= interval;
    // SpawnProjectile
}
```

✔ Vantaggi:
– semplice e intuitivo
– molto preciso anche in caso di micro-lag
– ottimo per cicli continui

# **Approccio 2 – Controllare il tempo dall’ultimo sparo (Time.time)**

Usiamo il tempo assoluto dall’avvio del gioco:

```csharp
if (Time.time - lastShotTime > interval)
{
    lastShotTime = Time.time;
    // SpawnProjectile
}
```

✔ Vantaggi:
– pulito
– ideale quando vuoi confrontare eventi nel tempo (hit, cooldown)

# **Spawn dei Proiettili**

In entrambi gli approcci, quando è il momento di sparare, si richiama il metodo che **instanzia** (crea) il proiettile.

Esempio generico:

```csharp
[SerializeField] private GameObject projectilePrefab;
[SerializeField] private Transform firePoint;

void SpawnProjectile()
{
    Instantiate(projectilePrefab, firePoint.position, firePoint.rotation);
}
```

### Il funzionamento:

* **projectilePrefab** = il proiettile da creare
* **firePoint** = un oggetto vuoto nella scena che indica la posizione e la direzione dello sparo
* **Instantiate** = crea un nuovo proiettile nella scena

# **Come si applica allo script del nemico?**

Ecco un esempio completo che unisce tutto:

```csharp
[SerializeField] private float interval = 1f;
[SerializeField] private GameObject projectilePrefab;
[SerializeField] private Transform firePoint;

private float timer;

void Update()
{
    timer += Time.deltaTime;

    if (timer > interval)
    {
        timer -= interval;
        SpawnProjectile();
    }
}

void SpawnProjectile()
{
    Instantiate(projectilePrefab, firePoint.position, firePoint.rotation);
}
```

# **Dettagli importanti**

* Il proiettile deve avere **rigidbody2D** e **collider2D** se vuoi farlo muovere e colpire.
* La direzione dello sparo dipende dal **firePoint.rotation**.
* Puoi aggiungere un componente **ProjectileController** al proiettile per gestirne velocità e danno.

---

# **Sfruttare l’Ereditarietà per gli Shooter**

Quando vogliamo far sparare sia il **giocatore** sia i **nemici**, possiamo evitare di duplicare il codice creando una struttura basata sull’**ereditarietà**.

Unity permette infatti di avere:

* una **classe base** con il codice comune
* classi derivate per comportamenti diversi (sparare in base al mouse, sparo costante, AI ecc.)

# **Creare una classe Base: BaseShooter**

La classe astratta **BaseShooter** (che estende `MonoBehaviour`) contiene solo gli elementi essenziali:

* un riferimento al **prefab del proiettile**
* una funzione **Shoot()**, che riceve:

  * **posizione** di spawn
  * **direzione** dello sparo

Questa classe non decide *quando* sparare:
si limita a *comportarsi come un generatore di proiettili*.

# **Classi derivate**

Si creano poi componenti specifiche che estendono BaseShooter:

### **A. PlayerInputShooter**

* ascolta l’input del giocatore
* spara verso la posizione del mouse
* usa `Input.mousePosition`

### **B. ConstantShooter**

* spara automaticamente ogni intervallo di tempo
* usa una logica interna (rate of fire costante)
* richiama periodicamente `Shoot()`

# **Come leggere la posizione del mouse in Unity**

* `Input.mousePosition` fornisce la posizione in **pixel dello schermo**
* per convertirla in una posizione nella scena, si usa:

```csharp
Camera.main.ScreenToWorldPoint(mousePosition)
```

⚠ **Attenzione:**
`mousePosition.z = 0`, ma la funzione si aspetta la distanza dal piano della Camera,
quindi bisogna impostare lo Z a un valore tra:

* `camera.nearClipPlane`
* `camera.farClipPlane`

# **Direzione dello sparo con il Transform**

Per decidere **posizione** e **direzione** del proiettile, si usa in genere un **oggetto figlio** (firePoint).

Ogni `Transform` ha **vettori direzionali locali**, quindi:

* `transform.forward` → asse Z+ (freccia blu), *poco usato in 2D*
* `transform.right` → asse X+ (freccia rossa)
* `transform.up` → asse Y+ (freccia verde)

In un 2D top-down, si usa quasi sempre:

* **transform.up** come direzione del proiettile

# **PlayerShooter e ConstantShooter nel loro Update()**

Entrambe le classi derivate, nel loro `Update()`, seguono la loro logica e poi chiamano:

```csharp
Shoot(firePoint.position, firePoint.up);
```

oppure nel caso del mouse:

```csharp
Vector3 dir = (mouseWorldPos - transform.position).normalized;
Shoot(firePoint.position, dir);
```

# **IN SINTESI**

* **BaseShooter** gestisce *come* sparare.
* Le classi derivate gestiscono *quando e verso dove* sparare.
* PlayerShooter usa il mouse → `ScreenToWorldPoint()`.
* ConstantShooter usa un timer → sparo automatico.
* La direzione di sparo deriva sempre da un vettore del `Transform`.

---




# **RIASSUNTO BREVE E SCHEMATIZZATO**

# **1. LifeController**

Gestisce HP e cosa accade a 0.

### **Enum comportamento**

* **DISABLE** → SetActive(false)
* **DESTROY** → Destroy(gameObject)
* **NONE** → niente

### **SetHP(int)**

* Clamp tra 0 e maxHP
* Se 0 → esegue enum

### **AddHP(int)**

* Chiama SetHP(GetHP() + amount)

### **Gestione MaxHP**

* Possibile autorisettare gli HP allo Start
* Decidere cosa succede se maxHP cambia
  (proporzione, clamp, ecc.)

---

# **2. Danno da Contatto**

Con **OnCollisionEnter2D / OnTriggerEnter2D**:

```csharp
if (collider.GetComponent<LifeController>() != null)
    life.AddHP(-damage);
```

---

# **3. Layers & LayerMask**

### **Layers**

* Organizzano oggetti (max 32)
* Un oggetto → 1 solo layer

### **Culling Mask (Camera)**

* Decide cosa la camera vede

### **Physics Collision Matrix**

* Decide quali layers collidono tra loro

### **LayerMask**

* Filtra nel codice (es. raycast)

---

# **4. Rate of Fire**

### **Metodo 1: deltaTime**

```csharp
timer += Time.deltaTime;
if (timer > interval) { timer -= interval; Shoot(); }
```

### **Metodo 2: Time.time**

```csharp
if (Time.time - lastShot > interval) { lastShot = Time.time; Shoot(); }
```

---

# **5. Spawn Proiettili**

```csharp
Instantiate(projectilePrefab, firePoint.position, firePoint.rotation);
```

* firePoint = punto/direzione di sparo
* projectile → rigidbody2D + collider2D

---

# **6. Ereditarietà per Shooter**

### **BaseShooter**

* contiene **Shoot(pos, dir)**
* contiene prefab

### **PlayerInputShooter**

* usa mouse
* `ScreenToWorldPoint()` per convertire la posizione
* attenzione allo Z del mouse

### **ConstantShooter**

* usa un timer → spara automaticamente

### **Direzioni Transform**

* **up** (Y+) → direzione più usata in 2D
* **right** (X+), **forward** (Z+) meno rilevanti

---

# **Super Sintesi**

* LifeController = gestione HP + morte
* Danni tramite collisioni
* Layers = organizzazione e filtraggio
* LayerMask = filtri nei raycast
* Rate of Fire = 2 modi per contare il tempo
* Spawn = Instantiate() sul firePoint
* Shooter = Base + Player/Macchina con ereditarietà

---

